# HW_3 - Web services 

In [1]:
import pandas as pd
import requests
import re

dests = pd.read_csv('C:/Users/YaelD/Desktop/dests.txt', header=None, names=['Target'])  # Importing the Data

def get_dis_dur(destination):
    # Returns distance and travel duration from TLV
    # In case of an illegal location - return None
    destinations = destination
    origins = 'Tel Aviv'
    units = 'kilometers'
    api_key = ''
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?destinations=%s&origins=%s&units=%s&key=%s" % (
        destinations, origins, units, api_key)
    try:
        response_data = requests.get(url)
        if not response_data.status_code == 200:
            print("HTTP error", response_data.status_code)
        else:
            try:
                response_Data = response_data.json()
                if not response_Data["status"] == "OK":
                    print('Error in Google API')
                else:
                    Distance_km = response_Data['rows'][0]['elements'][0]['distance']['value'] / 1000
                    Duration_sec = response_Data['rows'][0]['elements'][0]['duration']['value']
                    mins = (Duration_sec // 60) % 60
                    hours = (Duration_sec // 3600)
                    Duration = f"{hours} hours + {mins} mins"
                    return Distance_km, Duration
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return None, None

dests[['Distance_km', 'Duration (hours + minutes)']] = dests['Target'].apply(lambda x: pd.Series(get_dis_dur(x)))

def get_lat_lng(address_string):
    # Returns longitude and latitude of given location
    # In case of illegal location - return None
    address = address_string
    api_key = ''
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:
        response_data = requests.get(url)
        if not response_data.status_code == 200: 
            print("HTTP error",response_data.status_code)
        else:
            try:
                response_Data = response_data.json()
                if not response_Data["status"] == "OK":
                    print('Error in Google API')
                else:
                    Longitude = response_Data['results'][0]['geometry']['location']['lng']
                    Latitude = response_Data['results'][0]['geometry']['location']['lat']
                    
                    # longitude = (response_data2['results'][0]['geometry']['location']['lng'])
                    # latitude = (response_data2['results'][0]['geometry']['location']['lat'])
                    return Longitude, Latitude
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return None, None

dests[['Longitude', 'Latitude']] = dests['Target'].apply(lambda x: pd.Series(get_lat_lng(x)))


In [2]:
dests

,Target,Distance_km,Duration (hours + minutes),Longitude,Latitude
0,Istanbul,1815.227,21 hours + 2 mins,28.978359,41.008238
1,Amsterdam,4532.541,48 hours + 3 mins,4.904139,52.367573
2,Valletta,3792.883,50 hours + 52 mins,14.514100,35.899237
3,Basel,4092.805,44 hours + 2 mins,7.588576,47.559599
4,Doha,2164.477,22 hours + 38 mins,51.531040,25.285447


In [3]:
dests = dests.sort_values('Distance_km',ascending=False)
dests.head(3)

,Target,Distance_km,Duration (hours + minutes),Longitude,Latitude
1,Amsterdam,4532.541,48 hours + 3 mins,4.904139,52.367573
3,Basel,4092.805,44 hours + 2 mins,7.588576,47.559599
2,Valletta,3792.883,50 hours + 52 mins,14.514100,35.899237
